# **Natural Language Processing with RNNs and Attention**

In [1]:
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import tensorflow_datasets as tfds
import os
import tensorflow_hub as hub
if "google.colab" in sys.modules:
    %pip install -q -U transformers
    %pip install -q -U datasets
from transformers import pipeline
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.5 MB/s eta 0:00:00


In [2]:
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [3]:
shakespeare_url = "https://homl.info/shakespeare"  # shortcut URL
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1115394/1115394 [==============================] - 0s 0us/step


In [4]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character",
                                                   standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

encoded -= 2 
n_tokens = text_vec_layer.vocabulary_size() - 2  
dataset_size = len(encoded)  # total number of chars = 1,115,394



In [5]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [6]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True,
                       seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)



# **Char RNN Model**

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True)
history = model.fit(train_set, validation_data=valid_set, epochs=10,
                    callbacks=[model_ckpt])

Epoch 1/10
  31247/Unknown - 354s 11ms/step - loss: 1.3875 - accuracy: 0.5753

31247/31247 [==============================] - 372s 11ms/step - loss: 1.3875 - accuracy: 0.5753 - val_loss: 1.6129 - val_accuracy: 0.5337
Epoch 2/10
31246/31247 [============================>.] - ETA: 0s - loss: 1.2924 - accuracy: 0.5972

31247/31247 [==============================] - 338s 10ms/step - loss: 1.2924 - accuracy: 0.5972 - val_loss: 1.5876 - val_accuracy: 0.5395
Epoch 3/10
31245/31247 [============================>.] - ETA: 0s - loss: 1.2748 - accuracy: 0.6012

31247/31247 [==============================] - 343s 11ms/step - loss: 1.2748 - accuracy: 0.6012 - val_loss: 1.5725 - val_accuracy: 0.5422
Epoch 4/10
31247/31247 [==============================] - ETA: 0s - loss: 1.2651 - accuracy: 0.6034

31247/31247 [==============================] - 337s 10ms/step - loss: 1.2651 - accuracy: 0.6034 - val_loss: 1.5720 - val_accuracy: 0.5437
Epoch 5/10
31241/31247 [============================>.] - ETA: 0s - loss: 1.2587 - accuracy: 0.6046

31247/31247 [==============================] - 335s 10ms/step - loss: 1.2587 - accuracy: 0.6046 - val_loss: 1.5656 - val_accuracy: 0.5452
Epoch 6/10
31243/31247 [============================>.] - ETA: 0s - loss: 1.2540 - accuracy: 0.6056

31247/31247 [==============================] - 337s 10ms/step - loss: 1.2540 - accuracy: 0.6056 - val_loss: 1.5589 - val_accuracy: 0.5485
Epoch 7/10
31247/31247 [==============================] - 329s 10ms/step - loss: 1.2503 - accuracy: 0.6065 - val_loss: 1.5522 - val_accuracy: 0.5477
Epoch 8/10
31247/31247 [==============================] - 336s 10ms/step - loss: 1.2477 - accuracy: 0.6072 - val_loss: 1.5584 - val_accuracy: 0.5480
Epoch 9/10
31245/31247 [============================>.] - ETA: 0s - loss: 1.2454 - accuracy: 0.6076

31247/31247 [==============================] - 336s 10ms/step - loss: 1.2454 - accuracy: 0.6076 - val_loss: 1.5533 - val_accuracy: 0.5495
Epoch 10/10
31247/31247 [==============================] - 335s 10ms/step - loss: 1.2441 - accuracy: 0.6078 - val_loss: 1.5537 - val_accuracy: 0.5493


In [8]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2), 
    model
])

# **Generating Fake Shakespearean Text**

In [9]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])  # probas = 50%, 40%, and 10%
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)  # draw 8 samples


def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text


print(extend_text("To be or not to be", temperature=0.01))

print(extend_text("To be or not to be", temperature=1))

print(extend_text("To be or not to be", temperature=100))

1/1 [==============================] - 0s 23ms/step
To be or not to be a life
and i will not see her and the strange and
1/1 [==============================] - 0s 24ms/step
To be or not to beg,
in admiraw of confessor, rather louds thee:
ay,
1/1 [==============================] - 0s 25ms/step
To be or not to beevicm-vilv!?$ez?gmjz :3?ljb'va;!td&
i.ur3l'-j!3emq


# **Stateful RNN**

In [10]:
def to_dataset_for_stateful_rnn(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length + 1)).batch(1)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

stateful_train_set = to_dataset_for_stateful_rnn(encoded[:1_000_000], length)
stateful_valid_set = to_dataset_for_stateful_rnn(encoded[1_000_000:1_060_000],
                                                 length)
stateful_test_set = to_dataset_for_stateful_rnn(encoded[1_060_000:], length)

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16,
                              batch_input_shape=[1, None]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_stateful_shakespeare_model",
    monitor="val_accuracy",
    save_best_only=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(stateful_train_set, validation_data=stateful_valid_set,
                    epochs=10, callbacks=[ResetStatesCallback(), model_ckpt])



Epoch 1/10
   9995/Unknown - 76s 7ms/step - loss: 1.8655 - accuracy: 0.4513

9999/9999 [==============================] - 82s 8ms/step - loss: 1.8653 - accuracy: 0.4513 - val_loss: 1.7121 - val_accuracy: 0.4890
Epoch 2/10
9999/9999 [==============================] - ETA: 0s - loss: 1.5663 - accuracy: 0.5278

9999/9999 [==============================] - 80s 8ms/step - loss: 1.5663 - accuracy: 0.5278 - val_loss: 1.6270 - val_accuracy: 0.5112
Epoch 3/10
9998/9999 [============================>.] - ETA: 0s - loss: 1.4883 - accuracy: 0.5474

9999/9999 [==============================] - 80s 8ms/step - loss: 1.4882 - accuracy: 0.5474 - val_loss: 1.5907 - val_accuracy: 0.5256
Epoch 4/10
9993/9999 [============================>.] - ETA: 0s - loss: 1.4488 - accuracy: 0.5576

9999/9999 [==============================] - 81s 8ms/step - loss: 1.4487 - accuracy: 0.5576 - val_loss: 1.5668 - val_accuracy: 0.5326
Epoch 5/10
9998/9999 [============================>.] - ETA: 0s - loss: 1.4243 - accuracy: 0.5639

9999/9999 [==============================] - 81s 8ms/step - loss: 1.4243 - accuracy: 0.5639 - val_loss: 1.5475 - val_accuracy: 0.5376
Epoch 6/10
9995/9999 [============================>.] - ETA: 0s - loss: 1.4076 - accuracy: 0.5682

9999/9999 [==============================] - 81s 8ms/step - loss: 1.4075 - accuracy: 0.5682 - val_loss: 1.5387 - val_accuracy: 0.5377
Epoch 7/10
9993/9999 [============================>.] - ETA: 0s - loss: 1.3950 - accuracy: 0.5713

9999/9999 [==============================] - 80s 8ms/step - loss: 1.3950 - accuracy: 0.5713 - val_loss: 1.5294 - val_accuracy: 0.5421
Epoch 8/10
9994/9999 [============================>.] - ETA: 0s - loss: 1.3860 - accuracy: 0.5736

9999/9999 [==============================] - 80s 8ms/step - loss: 1.3860 - accuracy: 0.5736 - val_loss: 1.5312 - val_accuracy: 0.5428
Epoch 9/10
9999/9999 [==============================] - 78s 8ms/step - loss: 1.3786 - accuracy: 0.5754 - val_loss: 1.5261 - val_accuracy: 0.5405
Epoch 10/10
9995/9999 [============================>.] - ETA: 0s - loss: 1.3733 - accuracy: 0.5765

9999/9999 [==============================] - 80s 8ms/step - loss: 1.3732 - accuracy: 0.5765 - val_loss: 1.5224 - val_accuracy: 0.5450


# **Converting to Stateless Model**

In [12]:
stateless_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

stateless_model.build(tf.TensorShape([None, None]))

stateless_model.set_weights(model.get_weights())

shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    stateless_model
])

# **Sentiment Analysis**

In [13]:
raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name="imdb_reviews",
    split=["train[:90%]", "train[90%:]", "test"],
    as_supervised=True
)
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteUFHXHQ/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteUFHXHQ/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteUFHXHQ/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [14]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=20)

Epoch 1/20
704/704 [==============================] - 25s 32ms/step - loss: 0.6934 - accuracy: 0.4990 - val_loss: 0.6931 - val_accuracy: 0.5016
Epoch 2/20
704/704 [==============================] - 22s 30ms/step - loss: 0.6934 - accuracy: 0.5042 - val_loss: 0.6942 - val_accuracy: 0.5008
Epoch 3/20
704/704 [==============================] - 21s 30ms/step - loss: 0.6914 - accuracy: 0.5041 - val_loss: 0.6950 - val_accuracy: 0.4992
Epoch 4/20
704/704 [==============================] - 22s 31ms/step - loss: 0.6902 - accuracy: 0.5064 - val_loss: 0.6955 - val_accuracy: 0.5020
Epoch 5/20
704/704 [==============================] - 22s 31ms/step - loss: 0.6887 - accuracy: 0.5085 - val_loss: 0.6949 - val_accuracy: 0.5024
Epoch 6/20
704/704 [==============================] - 22s 30ms/step - loss: 0.6871 - accuracy: 0.5036 - val_loss: 0.6942 - val_accuracy: 0.5040
Epoch 7/20
704/704 [==============================] - 21s 30ms/step - loss: 0.6864 - accuracy: 0.5062 - val_loss: 0.6938 - val_accuracy:

# **With Masking**

In [15]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5
704/704 [==============================] - 27s 34ms/step - loss: 0.5349 - accuracy: 0.7257 - val_loss: 0.6751 - val_accuracy: 0.5396
Epoch 2/5
704/704 [==============================] - 22s 32ms/step - loss: 0.4121 - accuracy: 0.8124 - val_loss: 0.3265 - val_accuracy: 0.8548
Epoch 3/5
704/704 [==============================] - 23s 32ms/step - loss: 0.3043 - accuracy: 0.8705 - val_loss: 0.3340 - val_accuracy: 0.8524
Epoch 4/5
704/704 [==============================] - 22s 32ms/step - loss: 0.2796 - accuracy: 0.8843 - val_loss: 0.4297 - val_accuracy: 0.8144
Epoch 5/5
704/704 [==============================] - 22s 31ms/step - loss: 0.2626 - accuracy: 0.8927 - val_loss: 0.3365 - val_accuracy: 0.8592


In [17]:
inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
token_ids = text_vec_layer(inputs)
mask = tf.math.not_equal(token_ids, 0)
Z = tf.keras.layers.Embedding(vocab_size, embed_size)(token_ids)
Z = tf.keras.layers.GRU(128, dropout=0.2)(Z, mask=mask)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(Z)
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=10)

Epoch 1/10
704/704 [==============================] - 29s 34ms/step - loss: 0.5144 - accuracy: 0.7410 - val_loss: 0.3874 - val_accuracy: 0.8432
Epoch 2/10
704/704 [==============================] - 23s 32ms/step - loss: 0.3449 - accuracy: 0.8555 - val_loss: 0.3778 - val_accuracy: 0.8240
Epoch 3/10
704/704 [==============================] - 24s 34ms/step - loss: 0.2906 - accuracy: 0.8803 - val_loss: 0.3726 - val_accuracy: 0.8356
Epoch 4/10
704/704 [==============================] - 24s 34ms/step - loss: 0.2687 - accuracy: 0.8916 - val_loss: 0.3160 - val_accuracy: 0.8624
Epoch 5/10
704/704 [==============================] - 27s 38ms/step - loss: 0.2551 - accuracy: 0.8984 - val_loss: 0.3154 - val_accuracy: 0.8596
Epoch 6/10
704/704 [==============================] - 23s 32ms/step - loss: 0.2371 - accuracy: 0.9048 - val_loss: 0.3574 - val_accuracy: 0.8504
Epoch 7/10
704/704 [==============================] - 23s 32ms/step - loss: 0.2219 - accuracy: 0.9124 - val_loss: 0.3864 - val_accuracy:

In [19]:
text_vec_layer_ragged = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size, ragged=True)
text_vec_layer_ragged.adapt(train_set.map(lambda reviews, labels: reviews))
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer_ragged,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=10)

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_7/gru_6/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_7/gru_6/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_7/gru_6/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


704/704 [==============================] - 27s 35ms/step - loss: 0.5415 - accuracy: 0.7184 - val_loss: 0.5411 - val_accuracy: 0.7612
Epoch 2/10
704/704 [==============================] - 24s 34ms/step - loss: 0.3341 - accuracy: 0.8608 - val_loss: 0.3154 - val_accuracy: 0.8644
Epoch 3/10
704/704 [==============================] - 24s 34ms/step - loss: 0.2911 - accuracy: 0.8793 - val_loss: 0.3202 - val_accuracy: 0.8616
Epoch 4/10
704/704 [==============================] - 24s 34ms/step - loss: 0.2697 - accuracy: 0.8911 - val_loss: 0.3206 - val_accuracy: 0.8700
Epoch 5/10
704/704 [==============================] - 24s 34ms/step - loss: 0.2524 - accuracy: 0.8978 - val_loss: 0.4399 - val_accuracy: 0.8288
Epoch 6/10
704/704 [==============================] - 25s 35ms/step - loss: 0.2348 - accuracy: 0.9073 - val_loss: 0.3299 - val_accuracy: 0.8584
Epoch 7/10
704/704 [==============================] - 24s 34ms/step - loss: 0.2173 - accuracy: 0.9148 - val_loss: 0.3346 - val_accuracy: 0.8580
Epo

# **Pretrained Embeddings and Language Models**

In [20]:
os.environ["TFHUB_CACHE_DIR"] = "my_tfhub_cache"
tf.random.set_seed(42) 
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                   trainable=True, dtype=tf.string, input_shape=[]),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, validation_data=valid_set, epochs=10)

Epoch 1/10
704/704 [==============================] - 102s 124ms/step - loss: 0.3175 - accuracy: 0.8607 - val_loss: 0.2384 - val_accuracy: 0.8996
Epoch 2/10
704/704 [==============================] - 87s 123ms/step - loss: 0.0494 - accuracy: 0.9853 - val_loss: 0.3031 - val_accuracy: 0.8968
Epoch 3/10
704/704 [==============================] - 87s 124ms/step - loss: 0.0081 - accuracy: 0.9983 - val_loss: 0.4071 - val_accuracy: 0.8912
Epoch 4/10
704/704 [==============================] - 87s 123ms/step - loss: 0.0014 - accuracy: 0.9997 - val_loss: 0.4265 - val_accuracy: 0.8988
Epoch 5/10
704/704 [==============================] - 87s 124ms/step - loss: 5.5663e-04 - accuracy: 1.0000 - val_loss: 0.4667 - val_accuracy: 0.8964
Epoch 6/10
704/704 [==============================] - 87s 124ms/step - loss: 4.8191e-04 - accuracy: 1.0000 - val_loss: 0.4814 - val_accuracy: 0.8976
Epoch 7/10
704/704 [==============================] - 87s 123ms/step - loss: 4.4691e-04 - accuracy: 1.0000 - val_loss: 0.

# **Encoder–Decoder Network for Neural Machine Translation**


In [21]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets",
                               extract=True)
text = (Path(path).with_name("spa-eng") / "spa.txt").read_text()

import numpy as np

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.seed(42)
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)

for i in range(3):
    print(sentences_en[i], "=>", sentences_es[i])

vocab_size = 1000
max_length = 50
text_vec_layer_en = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_es = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

text_vec_layer_en.get_vocabulary()[:10]
text_vec_layer_es.get_vocabulary()[:10]

X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)

model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

2638744/2638744 [==============================] - 1s 0us/step
How boring! => Qué aburrimiento!
I love sports. => Adoro el deporte.
Would you like to swap jobs? => Te gustaría que intercambiemos los trabajos?
Epoch 1/10
3125/3125 [==============================] - 71s 20ms/step - loss: 0.4153 - accuracy: 0.4257 - val_loss: 0.3102 - val_accuracy: 0.5212
Epoch 2/10
3125/3125 [==============================] - 61s 20ms/step - loss: 0.2668 - accuracy: 0.5705 - val_loss: 0.2383 - val_accuracy: 0.6045
Epoch 3/10
3125/3125 [==============================] - 63s 20ms/step - loss: 0.2085 - accuracy: 0.6426 - val_loss: 0.2065 - val_accuracy: 0.6471
Epoch 4/10
3125/3125 [==============================] - 61s 20ms/step - loss: 0.1750 - accuracy: 0.6892 - val_loss: 0.1922 - val_accuracy: 0.6677
Epoch 5/10
3125/3125 [==============================] - 61s 19ms/step - loss: 0.1515 - accuracy: 0.7225 - val_loss: 0.1859 - val_accuracy: 0.6779
Epoch 6/10
3125/3125 [==============================] - 61s 2

# **Bidirectional RNNs**


In [22]:
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_state=True))

encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),  # short-term (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)]  # long-term (1 & 3)

decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 89s 25ms/step - loss: 0.3085 - accuracy: 0.5385 - val_loss: 0.2210 - val_accuracy: 0.6288
Epoch 2/10
3125/3125 [==============================] - 78s 25ms/step - loss: 0.1926 - accuracy: 0.6678 - val_loss: 0.1883 - val_accuracy: 0.6720
Epoch 3/10
3125/3125 [==============================] - 78s 25ms/step - loss: 0.1608 - accuracy: 0.7112 - val_loss: 0.1760 - val_accuracy: 0.6918
Epoch 4/10
3125/3125 [==============================] - 76s 24ms/step - loss: 0.1399 - accuracy: 0.7411 - val_loss: 0.1712 - val_accuracy: 0.6982
Epoch 5/10
3125/3125 [==============================] - 79s 25ms/step - loss: 0.1230 - accuracy: 0.7664 - val_loss: 0.1712 - val_accuracy: 0.7009
Epoch 6/10
3125/3125 [==============================] - 76s 24ms/step - loss: 0.1087 - accuracy: 0.7890 - val_loss: 0.1733 - val_accuracy: 0.7002
Epoch 7/10
3125/3125 [==============================] - 78s 25ms/step - loss: 0.0961 - accuracy: 0.8093 - val_loss: 0.1776 -

# **Attention Mechanisms**

In [23]:
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_sequences=True, return_state=True))

encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),  # short-term (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)]  # long-term (1 & 3)
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

attention_layer = tf.keras.layers.Attention()
attention_outputs = attention_layer([decoder_outputs, encoder_outputs])
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(attention_outputs)

model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 93s 26ms/step - loss: 0.3093 - accuracy: 0.5456 - val_loss: 0.2117 - val_accuracy: 0.6481
Epoch 2/10
3125/3125 [==============================] - 79s 25ms/step - loss: 0.1915 - accuracy: 0.6765 - val_loss: 0.1871 - val_accuracy: 0.6811
Epoch 3/10
3125/3125 [==============================] - 78s 25ms/step - loss: 0.1667 - accuracy: 0.7113 - val_loss: 0.1762 - val_accuracy: 0.6997
Epoch 4/10
3125/3125 [==============================] - 78s 25ms/step - loss: 0.1499 - accuracy: 0.7348 - val_loss: 0.1728 - val_accuracy: 0.7072
Epoch 5/10
3125/3125 [==============================] - 76s 24ms/step - loss: 0.1362 - accuracy: 0.7541 - val_loss: 0.1722 - val_accuracy: 0.7112
Epoch 6/10
3125/3125 [==============================] - 77s 25ms/step - loss: 0.1248 - accuracy: 0.7712 - val_loss: 0.1743 - val_accuracy: 0.7088
Epoch 7/10
3125/3125 [==============================] - 77s 25ms/step - loss: 0.1150 - accuracy: 0.7865 - val_loss: 0.1778 -

# **Attention Is All You Need:**

In [24]:
max_length = 50
embed_size = 128
pos_embed_layer = tf.keras.layers.Embedding(max_length, embed_size)
batch_max_len_enc = tf.shape(encoder_embeddings)[1]
encoder_in = encoder_embeddings + pos_embed_layer(tf.range(batch_max_len_enc))
batch_max_len_dec = tf.shape(decoder_embeddings)[1]
decoder_in = decoder_embeddings + pos_embed_layer(tf.range(batch_max_len_dec))

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_length, embed_size, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        assert embed_size % 2 == 0, "embed_size must be even"
        p, i = np.meshgrid(np.arange(max_length),
                           2 * np.arange(embed_size // 2))
        pos_emb = np.empty((1, max_length, embed_size))
        pos_emb[0, :, ::2] = np.sin(p / 10_000 ** (i / embed_size)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10_000 ** (i / embed_size)).T
        self.pos_encodings = tf.constant(pos_emb.astype(self.dtype))
        self.supports_masking = True

    def call(self, inputs):
        batch_max_length = tf.shape(inputs)[1]
        return inputs + self.pos_encodings[:, :batch_max_length]

pos_embed_layer = PositionalEncoding(max_length, embed_size)
encoder_in = pos_embed_layer(encoder_embeddings)
decoder_in = pos_embed_layer(decoder_embeddings)

# **Multi-Head Attention**

In [25]:
N = 2 
num_heads = 8
dropout_rate = 0.1
n_units = 128
encoder_pad_mask = tf.math.not_equal(encoder_input_ids, 0)[:, tf.newaxis]
Z = encoder_in
for _ in range(N):
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask=encoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    Z = tf.keras.layers.Dense(n_units, activation="relu")(Z)
    Z = tf.keras.layers.Dense(embed_size)(Z)
    Z = tf.keras.layers.Dropout(dropout_rate)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

decoder_pad_mask = tf.math.not_equal(decoder_input_ids, 0)[:, tf.newaxis]
causal_mask = tf.linalg.band_part(  # creates a lower triangular matrix
    tf.ones((batch_max_len_dec, batch_max_len_dec), tf.bool), -1, 0)

encoder_outputs = Z 
Z = decoder_in 
for _ in range(N):
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask=causal_mask & decoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=encoder_outputs, attention_mask=encoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    Z = tf.keras.layers.Dense(n_units, activation="relu")(Z)
    Z = tf.keras.layers.Dense(embed_size)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

Y_proba = tf.keras.layers.Dense(vocab_size, activation="softmax")(Z)
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 146s 44ms/step - loss: 0.2930 - accuracy: 0.5600 - val_loss: 0.2164 - val_accuracy: 0.6374
Epoch 2/10
3125/3125 [==============================] - 136s 44ms/step - loss: 0.2005 - accuracy: 0.6602 - val_loss: 0.1868 - val_accuracy: 0.6798
Epoch 3/10
3125/3125 [==============================] - 136s 44ms/step - loss: 0.1845 - accuracy: 0.6815 - val_loss: 0.1768 - val_accuracy: 0.6965
Epoch 4/10
3125/3125 [==============================] - 136s 43ms/step - loss: 0.1752 - accuracy: 0.6934 - val_loss: 0.1701 - val_accuracy: 0.7061
Epoch 5/10
3125/3125 [==============================] - 137s 44ms/step - loss: 0.1688 - accuracy: 0.7019 - val_loss: 0.1691 - val_accuracy: 0.7052
Epoch 6/10
3125/3125 [==============================] - 136s 44ms/step - loss: 0.1638 - accuracy: 0.7083 - val_loss: 0.1630 - val_accuracy: 0.7119
Epoch 7/10
3125/3125 [==============================] - 136s 43ms/step - loss: 0.1597 - accuracy: 0.7140 - val_loss: 0